In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
import data 
import pose
import models
import math

In [2]:
points_3D = np.array([

		(0, 0, 0), #fbl
        (0, 0, -3), #fbr
        (0, 3, 0),  #ftl
        (0, 3, -3), #ftr
        (9, 0, 0), #bbl
        (9, 0, -3), #bbr
        (9, 3, 0), #btl
        (9 , 3, -3), #btr
        (4.5,1.5,-1.5) #Center 
                     ])

In [3]:
dataset = data.coordsTrainingGenerator(2)
tmp = next(dataset)

[['8_img.png', '8_id.png', '8.txt'], ['6_img.png', '6_id.png', '6.txt'], ['2_img.png', '2_id.png', '2.txt'], ['4_img.png', '4_id.png', '4.txt'], ['0_img.png', '0_id.png', '0.txt'], ['9_img.png', '9_id.png', '9.txt'], ['5_img.png', '5_id.png', '5.txt'], ['7_img.png', '7_id.png', '7.txt']]
['8_img.png', '8_id.png', '8.txt']


In [4]:
tmp = next(dataset)

['6_img.png', '6_id.png', '6.txt']


In [5]:
img_num = 6
basePath = '../../unity/My project/dataset/train/'
mask = data.filePathToArray(basePath +str(img_num) + '_id.png', 600, 600)
for k in range(mask.shape[0]):
				for j in range(mask.shape[1]):

					if (mask[k][j]==np.array([0,0,0])).all():
						continue
					else:
						mask[k][j] = np.array([255,255,255])


In [ ]:
plt.imshow(tmp[0][0])
plt.imshow(mask)

In [6]:
#tmp er bilde og vektorfelt 
import random
population = np.where(mask > 0.99)[:2]
population = list(zip(population[0], population[1])) # y, x format
p1 = population.pop(random.randrange(len(population)))
print("før")
votes = data.ransacVoting(population,tmp[1][0])
print("etter")
mean = data.getMean(votes)
preds = data.dictToArray(mean)

matrix = np.array([[833.3333333, 0., 300], [0., 1250, 300], [0., 0., 1.]])
method = cv2.SOLVEPNP_ITERATIVE

_, R_exp, tVec = cv2.solvePnP(points_3D,
								preds,
								matrix,
								np.zeros(shape=[8, 1], dtype='float64'),
								flags=method)

R_exp = np.squeeze(R_exp)
tVec = np.squeeze(tVec)

før
etter


In [7]:
#Er dette kamerakoordinater eller koordinatsystemet definert ved punktene points_3D? 
#Må jo være kamerakoordinater? 
print(f'Image number: {img_num}')
print(f'Rotation vector: {R_exp}')
print(f'Translation vector: {tVec}')

Image number: 6
Rotation vector: [-0.12710367  0.27789779  0.10878306]
Translation vector: [-7.99522662 -2.940458   64.93274268]


In [ ]:
from scipy.spatial.transform import Rotation as R

In [ ]:
camera_rotation = np.array([0.1868, 0.2304, -0.02209562, 0.954718])
r2 = R.from_quat(camera_rotation)

#radianer
r_matrix = cv2.Rodrigues(R_exp)[0]


m = np.mat(r_matrix*r2.as_matrix())
mat = R.from_matrix(m)


print("--")
print(mat.as_quat())
print(m)

#cam_rot = eulerAnglesToRotationMatrix(camera_rotation)



total_rotation = R_exp

ZYX, jac = cv2.Rodrigues(total_rotation)

totalrotmax=np.array([[ZYX[0,0],ZYX[0,1],ZYX[0,2],tVec[0]],
[ZYX[1,0],ZYX[1,1],ZYX[1,2],tVec[1]],
[ZYX[2,0],ZYX[2,1],ZYX[2,2],tVec[2]],
[0,0,0,1]])


WtoC = np.mat(totalrotmax)
inverserotmax = np.linalg.inv(totalrotmax)

In [ ]:
np.matmul(r_matrix,r2.as_matrix())

In [ ]:
for i in q.as_euler('zxy'):
    print(i*180/math.pi)
#print(cam_rotation)

In [ ]:
inverserotmax

for line in inverserotmax:
    print ('  '.join(map(str, line)))

In [ ]:
print(cv2.Rodrigues(inverserotmax[0:3,0:3])[0])


#Kommet hit

In [ ]:
#Bruker ikke
points_3D_2 = np.array([

		(-3.4, 16.7, 82.3), #fbl
        (-3.4, 16.7, 79.3), #fbr
        (-6.4, 16.7, 82.3),  #ftl
        (-6.4, 16.7, 79.3), #ftr
        (-3.4, 25.8, 82.3), #bbl
        (-3.4, 25.8, 79.3), #bbr
        (-6.4, 25.8, 82.3), #btl
        (-6.4 , 25.8, 79.3), #btr
        (-4.9,21.3,80.8) #Center 
                     ])

In [ ]:
#rotation = data.pnp(points_3D ,preds, points_3D)

In [ ]:
#matrix = np.array([[572.4114, 0., 325.2611], [0., 573.57043, 242.04899], [0., 0., 1.]])


In [ ]:
# Calculates Rotation Matrix given euler angles.
def eulerAnglesToRotationMatrix(theta) :

    R_x = np.array([[1,         0,                  0                   ],
                    [0,         math.cos(theta[0]), -math.sin(theta[0]) ],
                    [0,         math.sin(theta[0]), math.cos(theta[0])  ]
                    ])

    R_y = np.array([[math.cos(theta[1]),    0,      math.sin(theta[1])  ],
                    [0,                     1,      0                   ],
                    [-math.sin(theta[1]),   0,      math.cos(theta[1])  ]
                    ])

    R_z = np.array([[math.cos(theta[2]),    -math.sin(theta[2]),    0],
                    [math.sin(theta[2]),    math.cos(theta[2]),     0],
                    [0,                     0,                      1]
                    ])

    R = np.dot(R_z, np.dot( R_y, R_x ))

    return R



In [ ]:
rot_matrix = rodrigues[0]
#cam_cord = np.append(tVec,1)
cam_cord = np.array([[np.squeeze(tVec[0])],[np.squeeze(tVec[1])],[np.squeeze(tVec[2])],[1]])

transform = np.array([[np.squeeze(tVec[0])],[np.squeeze(tVec[1])],[np.squeeze(tVec[2])]])
full_matrix = np.append(rot_matrix, transform, axis=1)
row = np.array([0,0,0,1])
final_matrix = np.vstack([full_matrix,row])

In [ ]:
np.matrix(final_matrix)*np.matrix([[0],[0],[-3],[1]])

In [ ]:
np.matrix([0,0,-3])

In [ ]:
matrix
final_matrix
one = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0]])
remp = np.matmul(matrix,one)
remp2 = np.matmul(remp,final_matrix)
uv = np.array([[0],[0],[1]])

In [ ]:
world_pos = np.array([-3.4, 16.7, 82.3])
test = np.matmul(rot_matrix,world_pos)
tVec

In [ ]:
np.squeeze(tVec) + test

In [ ]:
np.linalg.inv(rot_matrix)
uvPoint = np.array([300,300,1])

In [ ]:
tempMat = np.linalg.inv(rot_matrix)*np.linalg.inv(matrix)*uvPoint
tempMat2 = np.linalg.inv(rot_matrix)*tVec

In [ ]:
zConst = 0

In [ ]:
s = zConst + tempMat2[0,2]
s /= tempMat[0,2]

In [ ]:
wcPoint = np.linalg.inv(rot_matrix) *(s * np.linalg.inv(matrix) * uvPoint - np.squeeze(tVec))

In [ ]:
plotPoints, jacobian = cv2.projectPoints(np.array([(0.0,10.0,0.0)]), R_exp, tVec, matrix, np.zeros(shape=[8, 1], dtype='float64'))


In [ ]:
print(plotPoints)
preds[0]

In [ ]:

point1 = (int(preds[0][0]), 600-int(preds[0][1]))
 
point2 = (int(plotPoints[0][0][0]), 600-int(plotPoints[0][0][1]))
 

In [ ]:
for p in preds:
  cv2.circle(tmp[0][0], (int(p[0]), 600-int(p[1])), 3, (255,255,255), -1)
plt.imshow(tmp[0][0])

In [ ]:
cv2.line(tmp[0][0], point1, point2, (0,0,0), 2)
plt.imshow(tmp[0][0])